# 功課Week1-3：匯率爬蟲

請寫一個網頁爬蟲，截取臺灣銀行牌告匯率網頁：

http://rate.bot.com.tw/xrt?Lang=zh-TW


並且將資料用以下格式呈現在 Excel 内：
![](https://drive.google.com/uc?export=download&id=1YCl-QcAJCW951AhosB3HhuV7Fwy3hjMZ)

# Solution


### 1. Connect the web page of 'Bank Of Taiwan'.

In [1]:
import requests

url = "http://rate.bot.com.tw/xrt?Lang=zh-TW"
res = requests.get(url)
res.text

'<!DOCTYPE html>\r\n<html lang="zh-TW" class="no-js">\r\n<head>\r\n    <meta charset="utf-8" />\r\n    <title>臺灣銀行牌告匯率</title>\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n    <meta name="description" content="臺灣銀行匯率利率黃金牌價查詢">\r\n    <meta name="keywords" content="">\r\n    <meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=no">\r\n    <meta name="format-detection" content="telephone=no">\r\n        <meta name="robots" content="index,follow" />\r\n\r\n    <link rel="icon" type="image/x-icon" href="/favicon.ico">\r\n    <link rel="stylesheet" href="/Content/css/font-awesome.min.css">\r\n\r\n\r\n\r\n\r\n\r\n    <link rel="stylesheet" href="/Content/css/jquery-ui.min.css">\r\n    <link rel="stylesheet" href="/Content/main-3.3.0.css">\r\n    <style>\r\n        h1.hero__header_small {\r\n            font-size: 26px;\r\n            margin: 12px 0;\r\n        }\r\n\r\n            h1.hero__header_eng {\r\n                font-size: 20px;\r\n   

### 2. Parse the HTML with bs4.

In [4]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(res.text, "html.parser")
soup

<!DOCTYPE html>

<html class="no-js" lang="zh-TW">
<head>
<meta charset="utf-8"/>
<title>臺灣銀行牌告匯率</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="臺灣銀行匯率利率黃金牌價查詢" name="description"/>
<meta content="" name="keywords"/>
<meta content="width=device-width, initial-scale=1, user-scalable=no" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="index,follow" name="robots">
<link href="/favicon.ico" rel="icon" type="image/x-icon"/>
<link href="/Content/css/font-awesome.min.css" rel="stylesheet"/>
<link href="/Content/css/jquery-ui.min.css" rel="stylesheet"/>
<link href="/Content/main-3.3.0.css" rel="stylesheet"/>
<style>
        h1.hero__header_small {
            font-size: 26px;
            margin: 12px 0;
        }

            h1.hero__header_eng {
                font-size: 20px;
                margin: -8px 0;
            }

        .chart-key-value .sp {
            position: relative;
            top: 2px;
        }

   

### 3. Find the four currency. (USD HKD CNY JPY) 

一種貨幣佔一個row，一個row之內的所有`<td>`如下
```html
<tr>
    <td data-table="幣別" class="currency phone-small-font">
        <div>
            <div class="sp-div sp-america-div">
                <img title="幣別國旗" alt="幣別國旗" src="/Content/images/sprite_lateral.png" class="sp-img sp-america-img" />
            </div>
            <br class="visible-phone print_hide" />
            <div class="visible-phone print_hide">美金 (USD)</div>
            <div class="hidden-phone print_show" style="text-indent:30px;">美金 (USD)</div>
        </div>
    </td>
    <td data-table="本行現金買入" class="rate-content-cash text-right print_hide">29.695</td>
    <td data-table="本行現金賣出" class="rate-content-cash text-right print_hide">30.365</td>
    <td data-table="本行即期買入" class="rate-content-sight text-right print_hide" data-hide="phone">30.045</td>
    <td data-table="本行即期賣出" class="rate-content-sight text-right print_hide" data-hide="phone">30.145</td>
    <td data-table="遠期匯率買入/賣出" class="text-center print_hide phone-small-font"><a href="/xrt/forward/USD">查詢</a></td>
    <td data-table="歷史匯率" class="text-center print_hide phone-small-font"><a href="/xrt/history/USD" target="_blank">查詢</a></td>
    <td data-table="本行現金買入" class="text-right display_none_print_show print_width">29.695</td>
    <td data-table="本行現金賣出" class="text-right display_none_print_show print_width">30.365</td>
    <td data-table="本行即期買入" class="text-right display_none_print_show print_width">30.045</td>
    <td data-table="本行即期賣出" class="text-right display_none_print_show print_width">30.145</td>
</tr>
```

In [34]:
# Is there a way that no need 're'?
import re

def get_row(soup, currency):
    td = soup.findAll(text = re.compile(currency))[0].parent.parent.parent # first table data
    tr = td.parent # whole table row
    return tr

def get_price(row):
    cash_buy  = row.select('td[data-table="本行現金買入"]')[0].text
    cash_sale = row.select('td[data-table="本行現金賣出"]')[0].text
    spot_buy  = row.select('td[data-table="本行即期買入"]')[0].text
    spot_sale = row.select('td[data-table="本行即期賣出"]')[0].text
    price_dic = {"本行現金買入":cash_buy, "本行現金賣出":cash_sale , "本行即期買入":spot_buy, "本行即期賣出":spot_sale}    
    return price_dic

In [46]:
# Operations
currency_list = ["USD", "HKD", "CNY", "JPY"]
data = {}
for i in currency_list:
    row = get_row(soup, i)
    price_dic = get_price(row)
    data[i] = price_dic
    # print(i, price_dic)
data

{'USD': {'本行現金買入': '29.695',
  '本行現金賣出': '30.365',
  '本行即期買入': '30.045',
  '本行即期賣出': '30.145'},
 'HKD': {'本行現金買入': '3.719',
  '本行現金賣出': '3.923',
  '本行即期買入': '3.845',
  '本行即期賣出': '3.905'},
 'CNY': {'本行現金買入': '4.199',
  '本行現金賣出': '4.361',
  '本行即期買入': '4.271',
  '本行即期賣出': '4.321'},
 'JPY': {'本行現金買入': '0.266',
  '本行現金賣出': '0.2788',
  '本行即期買入': '0.2733',
  '本行即期賣出': '0.2773'}}

### 4. Write results in a csv

In [59]:
import csv

with open('output.csv', 'w', newline='') as csvfile:
    # 以 common 分隔欄位，建立 CSV 檔寫入器
    writer = csv.writer(csvfile, delimiter=',')
    
    # write title
    title_list = ['幣別', '本行現金買入', '本行現金賣出', '本行即期買入', '本行即期賣出']
    writer.writerow(title_list)
    
    # write content
    for currency in data:
        row = [currency]
        for key in data[currency]:
            row.append(data[currency][key])
            pass
        writer.writerow(row)

# Appendix: Download the full csv file. 
```html
<div class="text-center">
    <div class="btn-group">
        <a class="btn" href="/xrt/fltxt/0/day">下載文字檔</a>
        <a class="btn" href="/xrt/flcsv/0/day">下載 Excel (CSV) 檔</a>
        <button type="button" class="btn click_print">列印本頁</button>
        <button type="button" class="btn click_close">關閉本頁</button>
    </div>
</div>
```

In [18]:
# api url : https://rate.bot.com.tw/xrt/flcsv/0/day

btn_group = soup.select('.btn-group')[1]
url_link = btn_group.select('a')[0]['href']
dwn_link = "https://rate.bot.com.tw" + url_link
dwn_link

'https://rate.bot.com.tw/xrt/fltxt/0/day'

In [19]:
requests.get(dwn_link)

<Response [200]>

In [20]:
r = requests.get(dwn_link, stream=True)
open('download.csv', 'wb').write(r.content)

6311